In [13]:
import json
import os
import re
import google.generativeai as genai

# Configure the Gemini API
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel('gemini-1.5-pro-latest')

def read_file_lines(file_path):
    with open(file_path, 'r') as file:
        return [line.strip() for line in file if line.strip()]

def extract_json_from_response(response_text):
    # Try to extract JSON from triple backticks
    json_match = re.search(r'```(?:json)?\s*(\{.*?\})\s*```', response_text, re.DOTALL)
    if json_match:
        json_str = json_match.group(1)
    else:
        # If no triple backticks, assume the entire response is JSON
        json_str = response_text.strip()
    
    try:
        return json.loads(json_str)
    except json.JSONDecodeError:
        print(f"Error decoding JSON. Raw response: {response_text}")
        return None

def find_relevant_visuals(muscle_group, back_visuals, front_visuals):
    prompt = f"""
    Given the muscle group "{muscle_group}", find the most relevant visual(s) from the following lists:
    
    Back visuals: {json.dumps(back_visuals)}
    Front visuals: {json.dumps(front_visuals)}
    
    Return a JSON object with the following structure:
    {{
        "muscle_group": "name of the muscle group",
        "relevant_visuals": ["visual1", "visual2", ...]
    }}
    
    If no visuals are relevant, return an empty list for relevant_visuals.
    
    Example response:
    {{
        "muscle_group": "biceps brachii",
        "relevant_visuals": ["upper_arm_front", "forearm_front"]
    }}
    """
    
    response = model.generate_content(prompt)
    parsed_json = extract_json_from_response(response.text)
    
    if parsed_json is None:
        return {"muscle_group": muscle_group, "relevant_visuals": []}
    return parsed_json

def process_muscle_groups():
    muscle_groups = read_file_lines('../allExercises_muscles.txt')
    back_visuals = read_file_lines('../visuals_back.txt')
    front_visuals = read_file_lines('../visuals_front.txt')
    
    results = []
    
    for muscle_group in muscle_groups:
        result = find_relevant_visuals(muscle_group, back_visuals, front_visuals)
        results.append(result)
        print(f"Processed: {muscle_group}")
    
    with open('../muscle_group_visuals.json', 'w') as output_file:
        json.dump(results, output_file, indent=2)

    print("Processing complete. Results saved in muscle_group_visuals.json")


In [14]:
process_muscle_groups()

I0000 00:00:1723737516.816785 391955717 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Processed: abductors
Processed: adductors
Processed: anterior deltoid
Processed: biceps
Processed: brachioradialis
Processed: calves
Processed: erector spinae
Processed: forearm extensors
Processed: forearm flexors
Processed: gluteal muscles
Processed: hamstrings
Processed: lateral deltoid
Processed: latissimus dorsi
Processed: lower chest
Processed: obliques
Processed: posterior deltoid
Processed: pronators
Processed: quadriceps
Processed: rectus abdominis
Processed: supinators
Processed: transverse abdominis
Processed: trapezius
Processed: triceps
Processed: upper chest
Processing complete. Results saved in muscle_group_visuals.json
